In [1]:
import json

import os
import openai

from tqdm import tqdm
import time

import numpy as np
np.random.seed(0)

openai.api_key = os.getenv("OPENAI_API_KEY")

dev = json.load(open("../data/individual_dev_end2end_final.json"))

get_lemma = lambda x:" ".join([token.lemma_ for token in nlp(x)])


# nlp = spacy.load("en_core_web_sm")

relation_trigger = ['before', 'after', 'during', 'while']

warmup_qs = ['What will happen in the future?', 'What event has already finished?', 
             'What event has begun but has not finished?', 'What is happening now?',
             'What event has already happened?', 'What event has started?']

In [2]:
zero_shot_results = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
#     prompt = f"Given the context {context}, {question}" + ", select none or several from {" + ', '.join(all_events) + "}A:"
    
    zero_shot_results[key] = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              max_tokens=256,
              temperature=0
    )["choices"][0]["text"].strip()
    
    time.sleep(2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [1:18:26<00:00,  3.17s/it]


In [3]:
from eval_func_gpt3 import evaluate
evaluate(zero_shot_results, dev)

1483it [00:00, 27762.84it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.3980
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.4533
Eval on the current eval positive class Macro F1 (Agg) is: 0.3684
Eval on the current eval exact match (Agg) ratio is: 0.0539
Eval on the current eval exact match ratio (Relaxed) is: 0.0816
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0021
Eval on the current eval clustered EM (Relaxed) is: 0.0082
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0186


# one shot

In [4]:
train = json.load(open("../data/train_end2end_final.json"))

In [10]:
import numpy as np
np.random.seed(1)

random_key = np.random.choice(list(train.keys()), 1)[0]
print(random_key[0])
item = train[random_key]
print(item)

context = " ".join(item['context'])
question = item['question']
all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
examplar_prompt_0 = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
print(examplar_prompt_0)


d
{'answers': {'labels': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'context': ['A', 'public', 'security', 'bureau', 'official', 'said', 'the', 'police', 'were', 'unable', 'to', 'comment', 'on', 'the', 'statement', '.', 'But', 'Shanghai', 'police', 'did', 'say', 'that', 'on', 'March', '24', 'they', 'had', 'taken', 'similar', 'action', 'against', 'Bishop', 'Joseph', 'Fan', 'Zhongliang', ',', 'coadjutor', 'of', 'the', 'underground', 'Roman', 'Catholic', 'Church', '.'], 'question': 'What happened prior to the public security bureau official speaking?'}
Q: What happened prior to the public security bureau official speaking?, select none or several from {said, were, say, taken} 
A public security bureau official said the police were unable to comment on the st

In [26]:
with open('data/template_1_oneshot_exemplar_0.txt', 'w') as writer:
    writer.writelines(examplar_prompt_0)
with open('results/template_1_oneshot_0_pred', 'w') as writer:
    json.dump(one_shot_results_0, writer)

In [11]:
one_shot_results_0 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            one_shot_results_0[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompt_0 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [53:13<00:00,  2.15s/it]


In [14]:
evaluate(one_shot_results_0, dev)

1483it [00:00, 34989.50it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.4018
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.4344
Eval on the current eval positive class Macro F1 (Agg) is: 0.3548
Eval on the current eval exact match (Agg) ratio is: 0.0297
Eval on the current eval exact match ratio (Relaxed) is: 0.0452
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0000
Eval on the current eval clustered EM (Relaxed) is: 0.0021
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0041


In [12]:
import numpy as np
np.random.seed(2)

random_key = np.random.choice(list(train.keys()), 1)[0]
print(random_key)
item = train[random_key]
print(item)

context = " ".join(item['context'])
question = item['question']
all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
examplar_prompt_1 = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
print(examplar_prompt_1)


docid_wsj_0791_sentid_7_3N8OEVH1FV4K14LQJ53CXLQVB2SOOE_1
{'answers': {'labels': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]}, 'context': ['Norwood', 'is', 'controlled', 'by', 'Daniel', 'L.', 'Barnett', 'and', 'Paul', 'A.', 'Reese', ',', 'both', 'officers', 'of', 'Boston-based', 'Oasis', 'Capital', 'Management', 'Inc.', ',', 'a', 'small', 'Boston', 'money', 'management', 'firm', '.', 'Also', 'involved', 'in', 'the', 'group', 'is', 'Robert', 'F.', 'Angelo', ',', 'formerly', 'Phoenix', "'s", 'senior', 'vice', 'president', ',', 'field', 'operations', ',', 'who', 'left', 'Phoenix', 'at', 'the', 'beginning', 'of', 'October', '.'], 'question': 'What is happening now?'}
Q: What is happenin

In [13]:
one_shot_results_1 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            one_shot_results_1[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompt_1 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [51:41<00:00,  2.09s/it]


In [25]:
with open('data/template_1_oneshot_exemplar_1.txt', 'w') as writer:
    writer.writelines(examplar_prompt_1)
with open('results/template_1_oneshot_1_pred', 'w') as writer:
    json.dump(one_shot_results_1, writer)

In [15]:
evaluate(one_shot_results_1, dev)

1483it [00:00, 49770.78it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.4020
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.4470
Eval on the current eval positive class Macro F1 (Agg) is: 0.3630
Eval on the current eval exact match (Agg) ratio is: 0.0384
Eval on the current eval exact match ratio (Relaxed) is: 0.0600
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0000
Eval on the current eval clustered EM (Relaxed) is: 0.0000
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0082


In [17]:
import numpy as np
np.random.seed(3)

random_key = np.random.choice(list(train.keys()), 1)[0]
print(random_key)
item = train[random_key]
print(item)

context = " ".join(item['context'])
question = item['question']
all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
if len(ground ) == 0:
    ground = ['none']
examplar_prompt_2 = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
print(examplar_prompt_2)


docid_AFP_ENG_20051212.0486_sentid_6_3TYCR1GOTD84EQ65SING5PVWKW6ZLK_2
{'answers': {'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]}, 'context': ['About', '59', 'percent', 'of', 'eligible', 'Iraqis', 'cast', 'ballots', 'last', 'January', 'for', 'an', 'interim', 'parliament', ',', 'and', '64', 'percent', 'voted', 'in', 'the', 'October', '15', 'referendum', 'on', 'a', 'new', 'constitution', '.', 'The', 'US', 'envoy', 'said', 'that', 'unlike', 'the', 'runup', 'to', 'January', "'s", 'ballot', ',', '@', 'the', 'level', 'of', 'violence', 'has', 'dropped', 'my', 'almost', 'any', 'measure', 'over', 'the', 'past', 'few', 'days

In [18]:
one_shot_results_2 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            one_shot_results_2[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompt_2 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

 18%|████████████████████▉                                                                                             | 273/1483 [09:08<37:45,  1.87s/it]

Request failed due to server shutdown {
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Sat, 17 Jun 2023 08:21:54 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'text-davinci-003', 'openai-organization': 'user-7geflohwmbcawqczzwe5bb3a', 'openai-processing-ms': '1936', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '3000', 'x-ratelimit-limit-tokens': '250000', 'x-ratelimit-remaining-requests': '2999', 'x-ratelimit-remaining-tokens': '249744', 'x-ratelimit-reset-requests': '20ms', 'x-ratelimit-reset-tokens': '61ms', 'x-request-id': '241b7ae34174c03fb04031604db51a7f', 'CF-Cache-Status': 'DY

 22%|█████████████████████████                                                                                         | 326/1483 [11:04<38:07,  1.98s/it]

The server had an error while processing your request. Sorry about that!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1483/1483 [50:48<00:00,  2.06s/it]


In [20]:
evaluate(one_shot_results_2, dev)

1483it [00:00, 37852.52it/s]


Total 1483 questions
Eval on the current eval positive class Micro F1 (Agg) is: 0.3988
Eval on the current eval positive class Macro F1 (Relaxed) is: 0.4678
Eval on the current eval positive class Macro F1 (Agg) is: 0.3740
Eval on the current eval exact match (Agg) ratio is: 0.0937
Eval on the current eval exact match ratio (Relaxed) is: 0.1308
Eval on 485 Clusters
Eval on the current eval clustered EM (Agg) is: 0.0062
Eval on the current eval clustered EM (Relaxed) is: 0.0124
Eval on the current eval clusrered F1 (max>=0.8) is: 0.0309


In [24]:
with open('data/template_1_oneshot_exemplar_2.txt', 'w') as writer:
    writer.writelines(examplar_prompt_2)
with open('results/template_1_oneshot_2_pred', 'w') as writer:
    json.dump(one_shot_results_2, writer)

In [ ]:
# three-shot

In [ ]:
train = json.load(open("../data/train_end2end_final.json"))
import numpy as np
np.random.seed(1)

random_keys = np.random.choice(list(train.keys()), 3)
print(random_keys)
items = [train[k] for k in random_keys]
print(items)


examplar_list = []
for item in items:
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    if len(ground) == 0:
        ground = ['none']
    tmp_prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
    examplar_list.append(tmp_prompt)
examplar_threeshot_0 = "\n\n".join(examplar_list)


In [ ]:
three_shot_results_0 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            three_shot_results_0[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_threeshot_0 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

In [19]:
train = json.load(open("../data/train_end2end_final.json"))
import numpy as np
np.random.seed(2)

random_keys = np.random.choice(list(train.keys()), 3)
print(random_keys)
items = [train[k] for k in random_keys]
print(items)


examplar_list = []
for item in items:
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    if len(ground) == 0:
        ground = ['none']
    tmp_prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
    examplar_list.append(tmp_prompt)
examplar_threeshot_1 = "\n\n".join(examplar_list)


['docid_wsj_0791_sentid_7_3N8OEVH1FV4K14LQJ53CXLQVB2SOOE_1'
 'docid_AFP_ENG_19970417.0024_sentid_1_35K3O9HUAC2K335DAK65P2PU4IFFEA_5'
 'docid_AFP_ENG_20051215.0093_sentid_16_3J4Q2Z4UTZSBBSHD90B5N708JY9WQ2_0']
[{'answers': {'labels': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'types': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]}, 'context': ['Norwood', 'is', 'controlled', 'by', 'Daniel', 'L.', 'Barnett', 'and', 'Paul', 'A.', 'Reese', ',', 'both', 'officers', 'of', 'Boston-based', 'Oasis', 'Capital', 'Management', 'Inc.', ',', 'a', 'small', 'Boston', 'money', 'management', 'firm', '.', 'Also', 'involved', 'in', 'the', 'group', 'is', 'Robert', 'F.', 'Angelo', ',', 'formerly', 'Phoenix', "'s", 'senior', 'vice', 'president', ',', 'field',

In [ ]:
three_shot_results_1 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            three_shot_results_1[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_threeshot_1 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)

In [ ]:
train = json.load(open("../data/train_end2end_final.json"))
import numpy as np
np.random.seed(3)

random_keys = np.random.choice(list(train.keys()), 3)
print(random_keys)
items = [train[k] for k in random_keys]
print(items)


examplar_list = []
for item in items:
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    if len(ground) == 0:
        ground = ['none']
    tmp_prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + f"\nA: {', '.join(ground)}"
    examplar_list.append(tmp_prompt)
examplar_threeshot_2 = "\n\n".join(examplar_list)


In [ ]:
three_shot_results_2 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            three_shot_results_2[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_threeshot_2 + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)